In [1]:
import struct
import os
import scipy.io
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPool1D, Dropout

In [2]:
movie_feel = pd.read_csv('./movies.csv')
movie_feel

,First,Second,Third,Fourth,Fifth,Sixth
0,neutral,disgust,fear,sad,happy,anger
1,anger,disgust,neutral,happy,sad,fear
2,happy,neutral,anger,sad,disgust,fear
3,disgust,happy,sad,anger,neutral,fear
4,neutral,anger,disgust,happy,sad,fear
...,...,...,...,...,...,...
84,neutral,sad,happy,fear,disgust,anger
85,anger,sad,neutral,fear,happy,disgust
86,sad,anger,disgust,fear,neutral,happy
87,disgust,sad,anger,happy,fear,neutral


In [3]:
scores = pd.read_csv('./score.csv')
scores

,Video1_neutral,Video1_fear,Video1_sad,Video1_happy,Video1_anger,Video1_disgust,Video2_neutral,Video2_fear,Video2_sad,Video2_happy,...,Video5_sad,Video5_happy,Video5_anger,Video5_disgust,Video6_neutral,Video6_fear,Video6_sad,Video6_happy,Video6_anger,Video6_disgust
0,7,1,1,5,1,1,1,5,1,1,...,1,6,1,1,1,1,1,1,6,3
1,1,3,3,1,6,6,1,3,3,1,...,6,1,2,1,4,4,1,1,1,1
2,1,1,1,4,1,1,4,1,1,1,...,1,1,1,5,1,7,1,1,1,1
3,1,5,1,1,1,7,1,1,1,4,...,1,1,1,1,1,7,1,1,1,1
4,6,1,1,1,1,1,1,1,1,1,...,3,1,1,1,7,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,7,1,1,1,1,1,4,1,3,1,...,1,1,1,6,4,1,1,1,4,1
85,1,2,4,1,6,1,1,1,4,1,...,1,5,1,1,1,1,1,1,1,3
86,2,1,7,1,1,1,1,2,7,1,...,1,3,1,1,4,1,1,6,1,2
87,1,1,2,1,4,3,1,1,4,1,...,1,1,1,1,4,1,1,1,1,1


In [4]:
feel_order = pd.read_csv('./video_id.csv')
feeling = feel_order['Unnamed: 1']

mov_feel = []
for i in range(len(feeling)) :
    mov_feel.append(feeling[i])
mov_feel

['neutral',
 'disgust',
 'fear',
 'sad',
 'happy',
 'anger',
 'neutral',
 'anger',
 'disgust',
 'happy',
 'sad',
 'fear',
 'fear',
 'happy',
 'sad',
 'neutral',
 'disgust',
 'anger',
 'neutral',
 'anger',
 'sad',
 'disgust',
 'fear',
 'happy',
 'fear',
 'happy',
 'sad',
 'neutral',
 'anger',
 'disgust',
 'fear',
 'sad',
 'anger',
 'neutral',
 'happy',
 'disgust',
 'happy',
 'disgust',
 'fear',
 'anger',
 'sad',
 'neutral',
 'anger',
 'neutral',
 'fear',
 'sad',
 'disgust',
 'happy',
 'happy',
 'neutral',
 'anger',
 'fear',
 'sad',
 'disgust',
 'anger',
 'fear',
 'neutral',
 'disgust',
 'sad',
 'happy',
 'disgust',
 'sad',
 'neutral',
 'happy',
 'fear',
 'anger',
 'sad',
 'fear',
 'neutral',
 'happy',
 'disgust',
 'anger',
 'fear',
 'sad',
 'disgust',
 'happy',
 'neutral',
 'anger',
 'neutral',
 'disgust',
 'anger',
 'happy',
 'sad',
 'fear',
 'anger',
 'happy',
 'sad',
 'fear',
 'disgust',
 'neutral',
 'fear',
 'neutral',
 'disgust',
 'anger',
 'sad',
 'happy',
 'sad',
 'happy',
 'disg

In [5]:
train_ids = ['001', '005', '006', '009', '010', '011', '012', '013', '015', '021', '022', '023',
             '024', '026', '028', '031', '032', '033', '036', '038', '041', '042', '044',
             '045', '047', '053', '054', '055', '060', '063', '064', '066', '070', '072',
             '073', '075', '076', '077', '078']
test_ids = ['002', '003', '007', '017', '018', '019', '020', '025', '029', '030', '039', '043',
            '048', '052', '059', '065', '068', '074']

In [6]:
ppg_num = ['001', '002', '003', '005', '006', '007', '009', '010', '011', '012', '013', '015', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '036', '038', '039', '041', '042', '043', '044', '045', '047', '048', '052', '053', '054', '055', '059', '060', '062', '063', '064', '065', '066', '068', '070', '072', '073', '074', '075', '076', '077', '078']

In [8]:
train_label = []
test_label = []
cnt = 0
max_length = 0
for num in train_ids :
    for i in range (cnt, cnt+6) :
        ppg_path = os.path.join('./' + num + '_' + mov_feel[i] + '.csv')
        x = pd.read_csv(ppg_path)
        x.columns = ['data']
        train_label.append(x['data'].values)
        if max_length < len(train_label[cnt]) :
            max_length = len(train_label[cnt])
        cnt+=1
max_length

219063

In [9]:
max_length = 219063

padded_data = np.zeros((len(train_label), max_length))

for i, d in enumerate(train_label):
    padded_data[i, :len(d)] = d

print(padded_data.shape)
x_train = padded_data
print(x_train)

(234, 219063)
[[-15.03 -17.47 -20.   ...   0.     0.     0.  ]
 [ 34.53  43.76  51.28 ...   0.     0.     0.  ]
 [ 30.93  29.36  27.68 ...   0.     0.     0.  ]
 ...
 [ 83.7   71.31  50.73 ...   0.     0.     0.  ]
 [ 23.82 -15.56 -47.87 ...   0.     0.     0.  ]
 [-62.75 -45.9  -21.28 ...   0.     0.     0.  ]]


In [10]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(max_length, 1)))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model.add(tf.keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(Flatten())

batch_size = 1
num_samples = 246
features = []

for i in range(0, num_samples, batch_size):
    batch = x_train[i:i+batch_size]
    extracted_features = model.predict(batch)
    features.append(extracted_features)

1/1 [==============================] - 0s 66ms/step


1/1 [==============================] - 0s 63ms/step


ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [13]:
features

[array([[0.        , 0.18296728, 0.        , 0.20718832, 0.00186171,
         0.        , 0.        , 0.2527008 , 0.15996979, 0.00159073,
         0.        , 0.0883617 , 0.        , 0.24034667, 0.14891611,
         0.04162296, 0.        , 0.06624652, 0.        , 0.18529144,
         0.14113395, 0.02988974, 0.        , 0.        , 0.13764529,
         0.        , 0.16225418, 0.        , 0.        , 0.01256363,
         0.10191584, 0.        , 0.        , 0.        , 0.        ,
         0.00692637, 0.18013191, 0.11665496, 0.        , 0.09478468,
         0.        , 0.16698326, 0.        , 0.        , 0.06059938,
         0.15007755, 0.        , 0.0183463 , 0.18838437, 0.        ,
         0.22729528, 0.05119113, 0.02759666, 0.        , 0.1105245 ,
         0.0768273 , 0.3094223 , 0.17189816, 0.10940379, 0.17266908,
         0.        , 0.02019391, 0.1537818 , 0.10072179]], dtype=float32),
 array([[0.        , 0.25511563, 0.        , 0.2873077 , 0.00305974,
         0.        , 0.     

In [21]:
ppgcsv = pd.DataFrame(np.array(features).reshape(234, 64))

In [25]:
ppgcsv.to_csv('ppg_features.csv')